In [1]:
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.5/136.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.6 MB/s eta 0:00:00


In [2]:
!pip install langchain_openai

In [3]:
pip install -U langchain langchain-openai

In [ ]:
LANGSMITH_TRACING=True
LANGSMITH_ENDPOINT="https://api.smith.langchain.com"
LANGSMITH_API_KEY=""
LANGSMITH_PROJECT="RAGT08ManuelS"
OPENAI_API_KEY=""

In [7]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

··········


In [11]:
import os

# Establecer la variable de entorno USER_AGENT
os.environ["USER_AGENT"] = "RAGT08ManuelS"

In [12]:
# | output: false
# | echo: false

from langchain_openai import ChatOpenAI
import os
import getpass

# Get the OpenAI API key from the user.
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key:")

# Instantiate the ChatOpenAI object with the API key set in the environment.
llm = ChatOpenAI(model="gpt-4o-mini")

Enter your OpenAI API key:··········


In [13]:
# | output: false
# | echo: false

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [14]:
# | output: false
# | echo: false

from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [28]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

assert len(docs) == 1
print(f"Total characters: {len(docs[0].page_content)}")

Total characters: 43130


In [16]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
vector_store = InMemoryVectorStore(embeddings)
_ = vector_store.add_documents(documents=all_splits)

In [17]:
from langchain import hub
from langchain_core.documents import Document
from typing_extensions import List, TypedDict

prompt = hub.pull("rlm/rag-prompt")

class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

In [18]:
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}

def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

In [19]:
from langgraph.graph import START, StateGraph

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [20]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

In [29]:
# Definir un ID para el hilo
config = {"configurable": {"thread_id": "A1"}}

# Ejecutar consultas
input_message = "What is Task Decomposition?"

for step in graph.stream(
    {"question": input_message},
    stream_mode="values",
    config=config,
):
    print(step)

{'question': 'What is Task Decomposition?', 'context': [], 'answer': "I don't know."}
{'question': 'What is Task Decomposition?', 'context': [], 'answer': "I don't know."}
{'question': 'What is Task Decomposition?', 'context': [], 'answer': 'Task Decomposition is the process of breaking down a complex task or problem into smaller, more manageable parts or sub-tasks. This approach facilitates understanding, planning, and execution of tasks by addressing each component individually. It is commonly used in fields such as project management, computer science, and cognitive psychology.'}


In [30]:
print(f"Total documents loaded: {len(docs)}")
if docs:
    print(docs[0].page_content[:500])  # Imprimir los primeros 500 caracteres del primer documento

Total documents loaded: 1


      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In
